# Classification of Wood Species based on Images

The Forest Species Database – Microscopic contains 2,240 microscopic images of `112` different wood
species. The images have been captured through microscope optics and the spatial resolution of the
images is 1024 x 768 pixels. The total size of the data set is approx. 3.5 GB.

## 0. Title
Lappeenranta-Lahti University of Technology LUT

BM20A6100 Advanced Data Analysis and Machine Learning - Luento-opetus 6.9.2021-17.12.2021

 
Practical Assignment

Group D: 
Mikhail Farmakovskii, stnum: `000327770`, developer; 
Daniil Kunin, stnum: `000331753`, developer.

19.12.2021


## 1. Problem
Train a convolutional neural network (CNN), classify the images to the predetermined classes (wood species) and quantitatively evaluate the model performance

## 2. Data
https://web.inf.ufpr.br/vri/databases/forest-species-database-microscopic/

## 3. Evaluation
Quantitatively evaluate the model performance

## 4. Features

* There are 112 wood species, 20 images per species = 2,240 images. (112 classes)


## Adjust workspace

### Frameworks

In [1]:
# Import TensorFlow into jupyter
import tensorflow as tf
# Import Pandas into jupyter
import pandas as pd
# Same for numpy
import numpy as np
import datetime 

# others
import os # access to folder
from os import path

from matplotlib.pyplot import imread # read image
import matplotlib.pyplot as plt # visualization

from sklearn.model_selection import train_test_split # for data splitting
from sklearn.metrics import confusion_matrix, classification_report # confusion matrix

# Neural network tools
from keras.models import Sequential # Keras
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense # layers
from keras.optimizers import Adam # Optimizer

### Variables

In [2]:
# Change this image size if you want, default 220
image_size = 224 # new size of image
NUM_EPOCHS = 50 # № of epochs
BATCH_SIZE = 50 # Define the batch size = 50. The total number of training objects presented in one batch.
num_of_images = 2240 # Set the number of images to use for experimenting with model. Maximum - 2240

# Folder adjustments
cwd = os.getcwd()
cwd = cwd.replace('\\','/')

tensor_board_dir = cwd + "/logs" # This folder will contain logs of training
filenames_dir = cwd + "/data" # 112 folders with classes here
modeldir = cwd # The model will be saved here
loaded_model = cwd + "/WoodClassification.h5" # The model will be loaded from here

## 0. Data processing

### Load images and their labels

In [3]:
def get_filenames(folder):
    """
    The function appends file paths to list
    """
    dirs = os.listdir(folder) # list of folders with species
    filenames = [] # create an empty list
    labels = [] # list for 
    for i in range(0, len(dirs)):
        cur_dir = str(folder + "/" + dirs[i] + "/")
        #print(cur_dir)
        list_of_files = os.listdir(cur_dir) # list of files in current folder
        for j in range(0, len(list_of_files)):
            species = "'" + dirs[i]  + "'" # get true classes
            #print(species)
            QQ = eval(species)
            labels.append(QQ)
        
            files = "'" + cur_dir + str(list_of_files[j]) + "'"
            q = eval(files) 
            filenames.append(q) # append image paths to list 
            
    return filenames, labels

In [4]:
filenames, labels = get_filenames(filenames_dir)
filenames[:5]

['C:/Users/misha/Desktop/sample_project_1/data/001 Ginkgo biloba/00101.png',
 'C:/Users/misha/Desktop/sample_project_1/data/001 Ginkgo biloba/00102.png',
 'C:/Users/misha/Desktop/sample_project_1/data/001 Ginkgo biloba/00103.png',
 'C:/Users/misha/Desktop/sample_project_1/data/001 Ginkgo biloba/00104.png',
 'C:/Users/misha/Desktop/sample_project_1/data/001 Ginkgo biloba/00105.png']

In [5]:
labels[:5]

['001 Ginkgo biloba',
 '001 Ginkgo biloba',
 '001 Ginkgo biloba',
 '001 Ginkgo biloba',
 '001 Ginkgo biloba']

In [6]:
# See if number of labels matches the number of filenames
if len(labels) == len(filenames):
    print("Number of labels matches the number of filenames")
else:
    print("Number of labels does not matches the number of filenames")

Number of labels matches the number of filenames


In [7]:
labels = np.array(labels) # convert to numpy array
labels[:5]

array(['001 Ginkgo biloba', '001 Ginkgo biloba', '001 Ginkgo biloba',
       '001 Ginkgo biloba', '001 Ginkgo biloba'], dtype='<U31')

### Boolean array

In [8]:
# Find the unique labels values
unique_species = np.unique(labels)
unique_species[:5]

array(['001 Ginkgo biloba', '002 Agathis becarii',
       '003 Araucaria angustifolia', '004 Cephalotaxus drupacea',
       '005 Cephalotaxus harringtonia'], dtype='<U31')

In [9]:
len(unique_species) # 112 - number of classes

112

In [10]:
# Turn every label into an array of boolens
# This code helps us to understand which class corresponds with label (True if yes, else no).
boolean_classes = [label == unique_species for label in labels]
boolean_classes[0]

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False])

### Data Splitting

In [11]:
# Setup x & y variables
x = filenames
y = boolean_classes

In [12]:
x[:2]

['C:/Users/misha/Desktop/sample_project_1/data/001 Ginkgo biloba/00101.png',
 'C:/Users/misha/Desktop/sample_project_1/data/001 Ginkgo biloba/00102.png']

In [13]:
y[:1]

[array([ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False])]

In [14]:
# Splitting
x_train, x_val, y_train, y_val = train_test_split(x[:num_of_images],
                                                  y[:num_of_images],
                                                  test_size=0.2)

len(x_train), len(y_train), len(x_val), len(y_val) 

(1792, 1792, 448, 448)

## 1.Preprocessing Images

### In this section we should convert images to tensors
`Tensors` are multi-dimensional arrays. Its type is np.array



In [15]:
# It is possible to convert pixels to array. Its shape is (768, 1024, 3)
image = imread(filenames[0])
image.shape

(768, 1024, 3)

In [16]:
image[:1]

array([[[0.28235295, 0.20784314, 0.15294118],
        [0.28235295, 0.19607843, 0.14509805],
        [0.32156864, 0.19215687, 0.13333334],
        ...,
        [0.1254902 , 0.10980392, 0.11372549],
        [0.09803922, 0.1254902 , 0.09803922],
        [0.10196079, 0.12156863, 0.10588235]]], dtype=float32)

### Function for color scaling and size reshaping 

In [17]:
def process_image(Image_path, img_size=image_size):
    """
    Here we are using a Tensorflow function to process images.
    Input: file path
    Output: Tensor (array)
    """
    
    # Read image by using path
    Image = tf.io.read_file(Image_path)
    # Turn the image into numerical tensor with 3 colour channels
    Image = tf.image.decode_png(Image, channels=3)
    # Convert the colour channel 0-255 to 0-1 
    Image = tf.image.convert_image_dtype(Image, tf.float32)
    # Resize the image
    Image = tf.image.resize(Image, size=[img_size, img_size])                                
    return Image

In [18]:
# Function to get image and its label
def get_image_label(image_path, label):
    """
    Function get_image_label returns the image after processing (Tensor type) & its label
    """
    image = process_image(image_path)
    return image, label

### Creating data samples
Getting many pairs by using function `get_image_label`

In [19]:
# Create a function to return data into batches
def create_data_batches(x, y=None, batch_size=BATCH_SIZE, valid_data=False, test_data=False):
    """
    Create batches of data. Pairs are used (tensor and true class)
    Input: x: image path, y: true class, batch size (default 32), also for test and validation data.
    Output: samples of data with tensors and their classes
    
    from_tensor_slices - creates a Dataset whose elements are slices of the given tensors.
    The given tensors are sliced along their first dimension.
    
    .batch - combines consecutive elements of this dataset into batches.
    """


  # If the data is a test dataset (no labels)
    if test_data:
        print("Creating test data batches.......")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(x))) # 
        data_batch = data.map(process_image).batch(BATCH_SIZE)
        return data_batch


  # If data is a valid dataset. Don't need to shuffle
    elif valid_data:
        print("Creating validation data batches.......")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(x), # filepaths
                                               tf.constant(y))) # labels
        data_batch = data.map(get_image_label).batch(BATCH_SIZE)
        return data_batch

    # this is for train data
    else:
        print("Creating training data batches.......")
        # Turn filepaths and labels into tensors
        data = tf.data.Dataset.from_tensor_slices((tf.constant(x),
                                               tf.constant(y)))
        # Shuffling pathnames and labels
        data = data.shuffle(buffer_size=len(x))

        data_batch = data.map(get_image_label).batch(BATCH_SIZE)

        return data_batch

In [20]:
# Create training and validation data batches
train_data = create_data_batches(x_train, y_train)
valid_data = create_data_batches(x_val, y_val, valid_data=True)

Creating training data batches.......


KeyboardInterrupt: 

### Visualizing Data Batches

In [ ]:
train_data

In [ ]:
# FUnction for viewing images in a data batch
def show_25_images(images, labels):
    """
    Displays a plot of 25 images and their labels from a data batch
    """
  
   # Setup the figure
    plt.figure(figsize=(18,18))
    # Loop through 25
    for i in range(10):
    # Create sublots (5 row, 5 columns)
        ax = plt.subplot(5 , 5, i+1)
        # Display an image
        plt.imshow(images[i])
        # Add the image label as the title
        plt.title(unique_species[labels[i].argmax()])
        # Grid lines off
        plt.axis("off")

In [ ]:
train_images, train_labels = next(train_data.as_numpy_iterator()) # inverse conversion to image
len(train_images), len(train_labels)

In [ ]:
# Visualize the data in a training batch
show_25_images(train_images, train_labels)

## 2. Building a model

In [ ]:
# Setup shape to the model
INPUT_SHAPE = [None, image_size, image_size, 3] # batch, heigh....

# Setup output shape of model
OUTPUT_SHAPE = len(unique_species)

In [ ]:
(INPUT_SHAPE[1:], OUTPUT_SHAPE)

### Setup the model

In [ ]:
model = None

In [ ]:
# Setup the model
# model = Sequential([
# Conv2D(16, (3, 3), activation='relu', input_shape=INPUT_SHAPE[1:]), 
# MaxPooling2D(2, 2),
# Conv2D(32, (3, 3), activation='relu'), MaxPooling2D(2, 2),
# MaxPooling2D(2, 2),
# Conv2D(128, (3, 3), activation='relu'),
# MaxPooling2D(2, 2),
# Flatten(),
# Dense(512, activation='relu'),
# Dropout((0.2)),
# Dense(512, activation='relu'),
# Dense(112, activation='softmax')
# ])

model = Sequential([
Conv2D(64, (1, 1), activation='relu', input_shape=INPUT_SHAPE[1:]),
Conv2D(64, (1, 1), activation='relu'),
MaxPooling2D(2, 2),
    
Conv2D(128, (1, 1), activation='relu'),
Conv2D(128, (1, 1), activation='relu'), 
MaxPooling2D(2, 2),
    
    
Conv2D(256, (1, 1), activation='relu'), 
Conv2D(256, (1, 1), activation='relu'), 
Conv2D(256, (1, 1), activation='relu'),
MaxPooling2D(2, 2),

        
Conv2D(512, (1, 1), activation='relu'), 
Conv2D(512, (1, 1), activation='relu'), 
Conv2D(512, (1, 1), activation='relu'), 
MaxPooling2D(2, 2),
    
Conv2D(512, (1, 1), activation='relu'), 
Conv2D(512, (1, 1), activation='relu'), 
Conv2D(512, (1, 1), activation='relu'),  
MaxPooling2D(2, 2),    
   
Flatten(),
Dense(4096, activation='relu'),

Dense(112, activation='softmax')
])

# Compile the model
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
optimizer=tf.keras.optimizers.Adam(),
metrics=["accuracy"])

### Layers of neural network

In [ ]:
model.summary()

### Settings for neural network

In [ ]:
# Load Tensorboard notebook extension
%load_ext tensorboard

In [ ]:
# Function for TensorBoard callback
def create_tensorboard_callback(tensor_board_dir):
    # Create a log directory for logs of training
    logdir = os.path.join(tensor_board_dir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    return tf.keras.callbacks.TensorBoard(logdir)

In [ ]:
print("GPU", "available" if tf.config.list_physical_devices("GPU") else "not available")

### Training

In [ ]:
# Function to save a model
def save_model(model, modeldir, suffix=None):
    """
    Saves a model in a directory 
    """
    model_path = modeldir + suffix + ".h5"
    print(f"Saving model to: {model_path}.......")
    model.save(model_path)
    return model_path

In [ ]:
# fucntion to load a model
def load_model(model_path):
    """
    Load a saved model
    """
    print(f"Loading saved model from: {model_path}.......")
    model = tf.keras.models.load_model(model_path)
                                    
    return model

In [ ]:
# Function to train
def train_model(model,tensor_board_dir):
    """
    Trains a given model and returns the trained version
    """
    # Create new tensorBoard session everytime we train a model
    tensorboard = create_tensorboard_callback(tensor_board_dir)
  # Fit the model
    model.fit(x=train_data, # images(tensors) and its labels(bool)
            epochs=NUM_EPOCHS,
            validation_data=valid_data,
            validation_freq=1,
            callbacks=[tensorboard])
    save_model(model,modeldir, suffix="/WoodClassification")
    return model

In [ ]:
if os.path.exists(loaded_model):
    model = load_model(loaded_model)
else:
    model = train_model(model,tensor_board_dir)

## 3. Evaluation

In [ ]:
predictions = model.predict(valid_data, verbose=1)
model.evaluate(valid_data)

In [ ]:
%tensorboard --logdir tensor_board_dir 

In [ ]:
# predictions[0]

In [ ]:
# unique_species

In [ ]:
# # First prediction
# index = 0
# print(predictions[index])
# print(f"Max value (prabability of prediction 1 image): {np.max(predictions[index])}")
# print(f"Sum:{np.sum(predictions[index])}")
# print(f"Max index:{np.argmax(predictions[index])}")
# print(f"Predicted label: {unique_species[np.argmax(predictions[index])]}")

### Probabilities

In [ ]:
def get_pred_label(predictions_probabilities):
    """
    Turns an array of prediciton into a labels
    """
    return unique_species[np.argmax(predictions_probabilities)]

In [ ]:
# Function to unbatch
def unbatchify(data):
    """
    Takes a batched dataset of (image, label) tensors and returns separate arrays
    of images and labels
    """
    images = []
    labels = []
    # loop through unbatched data
    for image, label in data.unbatch().as_numpy_iterator():
        images.append(image)
        labels.append(unique_species[np.argmax(label)])
    return images, labels

In [ ]:
val_images, val_labels = unbatchify(valid_data)
val_labels[:2]

In [ ]:
images_ = []
labels_ = []

# loop through unbatched data
for image, label in valid_data.unbatch().as_numpy_iterator():
    images_.append(image)
    labels_.append(label)

In [ ]:
def plot_pred_conf(prediction_probabilities, labels, n=1):
    """
    Plus the top 10 highest predictions confidences along with the truth label for n sample.
    """

    pred_prob, true_label = prediction_probabilities[n], labels[n]

    # Get the predicted label
    pred_label = get_pred_label(pred_prob)

    # Find the top 10 prediction indexes
    top_10_pred_indexes = pred_prob.argsort()[-10:][::-1]


    # Find the top 10 prediction confidence values
    top_10_pred_values = pred_prob[top_10_pred_indexes]

    # Find the top 10 prediction labels
    top_10_pred_labels = unique_species[top_10_pred_indexes]

    # Plotting
    top_plot = plt.bar(np.arange(len(top_10_pred_labels)),
                     top_10_pred_values,
                     color="grey")
    plt.xticks(np.arange(len(top_10_pred_labels)),
             labels=top_10_pred_labels,
             rotation="vertical")
  
      # Change the color of true labels. Green for true value.
    if np.isin(true_label, top_10_pred_labels):
        top_plot[np.argmax(top_10_pred_labels == true_label)].set_color("green")
    else:
        pass

In [ ]:
plot_pred_conf(prediction_probabilities=predictions,
               labels = val_labels,
               n=9)

### Classification report

In [ ]:
y_pred = []
for i in range(0, len(predictions)):
    y_pred.append(unique_species[np.argmax(predictions[i])])
y_pred[:2]

In [ ]:
print(classification_report(val_labels, y_pred))